# Multi-variate time series classification using InceptionTime

In this notebook, we use InceptionTime to perform for multi-variate time series classification by deep learning.

In [1]:

import  os,warnings, sys, shutil, os, numpy as np, pandas as pd
# import pandas as pd


warnings.filterwarnings("ignore")
# import pickle
# import torch
from sklearn.metrics import  accuracy_score
path = os.path.dirname(os.getcwd())

sys.path.insert(0,path + '/mutual_functions/' )
data_path = path + '/mutual_functions/data/' 
from usefull_functions import *
from evaluation_functions import *
# from plot_functions import *


from utils import rocket_classify,rocket_transform,predict,load_data_sktime,dl_model


# reload packages with new changes
%load_ext autoreload
%autoreload 2

# data_path
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
# torch.cuda.empty_cache()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
from sktime.classification.deep_learning.inceptiontime import InceptionTimeClassifier
import numpy as np
import seaborn as sns


In [2]:

prop = {'dataset':'AF', #'shortTracks_seperate.pkl
        'data_path' : data_path,
        
        
        
        'norm':'without_norm', #'MinMax01',MinMax-11 # 'mean_standardize', 'without_norm'
        'seq_len':100,#20, 
        'LEN':None, #None,#7
        'consecutive': True,

        'shuffle': False,
        'test_size' :0.2,
       

        'multivariate':True,


        # # multivariate version for multivariate time series input: Multivariate compiles via Numba on import.
        # 'transform': 'MiniRocket', #Rocket, 'MultiRocket','MiniRocket','MultiRocketMultivariate',  'MiniRocketMultivariate', 
        # # RidgeClassifierCV for smaller datasets (<20k training examples). For larger datasets, use logistic regression trained with stochastic gradient descent SGDClassifier(loss='log').
        # 'classifier': 'sgd', #'ridge','sgd'

        'model': 'InceptionTime', #CNNNetwork, FCNNetwork
        # cnn, fcn, resnet, inception, inceptiontime, mcdcnn, tlenet, encoder
        # 'InceptionTime', MLP, ResNet, MCDCNN (Multi Channel Deep Convolutional Neural Network)


}

In [3]:
from usefull_functions import *
from evaluation_functions import *


    
# X_train, y_train, X_test, y_test,_,_ = loadData(prop,prop['data_path'])
# # x_train, y_train, x_test, y_test = data_loader(data_path + prop['dataset'] + '/')
# # (X_train ==x_train).all()


# X_train,X_test = convert(X_train.copy()),convert(X_test.copy())
# y_train,y_test = y_train.astype(str),y_test.astype(str)
X_train, y_train, X_test, y_test =  load_data_sktime(multivariate = True)

data dimensions: train - (40, 6), test - (40, 6) with series length 100


# Load a dataset
The Basic Motions dataset, from [timeseriesclassification.com](http://www.timeseriesclassification.com), has time series in six dimensions.

In [4]:
# X_train, y_train = load_basic_motions(split="train", return_X_y=True)
# X_test, y_test = load_basic_motions(split="test", return_X_y=True)
# print(X_train.shape)
# print(X_test.shape)
# X_train.head()

# Train a deep neural network classifier
Here we choose to use the InceptionTime classifier. Other classifiers provided by sktime-dl include MLP, ResNet, MCDCNN (Multi Channel Deep Convolutional Neural Network)

In [7]:
from utils import *
model_types = ['InceptionTime','FCN','CNN','LSTMFCN', 'MLP','TapNet','MACNN','MCDCNN','ResNet' , 'SimpleRNN']#
for m in model_types:
    model = dl_model(model_type = m,n_epochs=1,verbose=False)

    print('model:', model.fit(X_train, y_train))
    print('score: ', model.score(X_test, y_test))

model: InceptionTimeClassifier(n_epochs=1)
1/1 [==============================] - 0s 319ms/step
score:  0.25
model: FCNClassifier(n_epochs=1)
3/3 [==============================] - 0s 6ms/step
score:  0.2
model: CNNClassifier(n_epochs=1)
3/3 [==============================] - 0s 1ms/step
score:  0.25
model: LSTMFCNClassifier(callbacks=[], n_epochs=1, verbose=False)
1/1 [==============================] - 0s 321ms/step
score:  0.25
model: MLPClassifier(n_epochs=1)
3/3 [==============================] - 0s 2ms/step
score:  0.225
model: TapNetClassifier(n_epochs=1)
3/3 [==============================] - 1s 54ms/step
score:  0.35
model: MACNNClassifier(n_epochs=1)
10/10 [==============================] - 0s 18ms/step
score:  0.25
model: MCDCNNClassifier(n_epochs=1)
3/3 [==============================] - 0s 2ms/step
score:  0.325
model: ResNetClassifier(n_epochs=1)
3/3 [==============================] - 0s 11ms/step
score:  0.25


model: SimpleRNNClassifier(num_epochs=1)
40/40 [==============================] - 0s 2ms/step
score:  0.225


# Grid Search
sktime-dl is compatible with scikit-learn and can use sklearn's GridSearchCV.

Here we search over two parameters, CNN kernel size and pooling size.

In [9]:
param_grid = {"nb_filters": [16, 32], "bottleneck_size": [16, 32]}
grid = GridSearchCV(network, param_grid=param_grid, cv=5)
# commented fit call to pass checks (meet time constraints)
# grid.fit(X_train, y_train)

# print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
# print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
# print("Best parameters: {}".format(grid.best_params_))